In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|adultinternetusag...|      false|
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db|individualsactivi...|      false|
| tabd_db|individualsactivi...|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|internetusagefreq...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-----------+



In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.P_GlobalSocialMediaUsage 
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.P_GlobalSocialMediaUsage  (
        region VARCHAR(45),
        social_media_users DOUBLE,
        year INT,
        mobile_penetration_percentage INT
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/P_GlobalSocialMediaUsage/'
    """
)

DataFrame[]

In [4]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|adultinternetusag...|      false|
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db|individualsactivi...|      false|
| tabd_db|individualsactivi...|      false|
| tabd_db| internetactivities1|      false|
| tabd_db| internetactivities2|      false|
| tabd_db|       internetusage|      false|
| tabd_db|internetusagefreq...|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|p_dailyinternetus...|      false|
| tabd_db|p_globalsocialmed...|      false|
| tabd_db|parentawarenessof...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|socialmediausersa...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-----------+



In [5]:
# Let's look into HDFS
spark.sql(
    """
    SELECT *
    FROM tabd_db.P_GlobalSocialMediaUsage
    """
).show()

+------+------------------+----+-----------------------------+
|region|social_media_users|year|mobile_penetration_percentage|
+------+------------------+----+-----------------------------+
+------+------------------+----+-----------------------------+



In [6]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.P_GlobalSocialMediaUsage
    """
).toPandas()

,col_name,data_type,comment
0,region,varchar(45),None
1,social_media_users,double,None
2,year,int,None
3,mobile_penetration_percentage,int,None
4,,,
5,# Detailed Table Information,,
6,Database,tabd_db,
7,Table,p_globalsocialmediausage,
8,Owner,jovyan,
9,Created Time,Mon Jun 07 17:48:46 UTC 2021,


In [7]:
# Let's put the files into HDFS

spark.sql(
    """
    SELECT *
    FROM tabd_db.P_GlobalSocialMediaUsage
    """
).toPandas()

,region,social_media_users,year,mobile_penetration_percentage


In [8]:
# recover partitions is needed so that the Hive Metastore (Catalog)
# is updated. Otherwise Hive and the querying engines do not know
# that there are new parittions in the partitioned table.

spark.sql(
    """
    SELECT *
    FROM tabd_db.P_GlobalSocialMediaUsage
    """
).show()

+------+------------------+----+-----------------------------+
|region|social_media_users|year|mobile_penetration_percentage|
+------+------------------+----+-----------------------------+
+------+------------------+----+-----------------------------+



In [9]:
spark.stop()